Setup and Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings

warnings.filterwarnings('ignore')

sns.set_style('whitegrid')
plt.rcParams['figure.dpi'] = 120

output_dir = Path("outputs")
figures_dir = Path("figures")

if not output_dir.exists():
    output_dir.mkdir()
if not figures_dir.exists():
    figures_dir.mkdir()

print("Environment setup complete.")


Environment setup complete.


Loading Data and extracting daily/hourly blocks

In [2]:
import pandas as pd
from pathlib import Path

# Load the workbook
excel_path = Path("data/Data-Analytics-Assignment.xlsx")
workbook = pd.ExcelFile(excel_path)


def extract_block(raw_data, block_label):
    """Extracts the specified data block (Daily/Hourly) from a given worksheet."""
    match_row = None
    for idx in range(raw_data.shape[0]):
        row_values = raw_data.iloc[idx].astype(str).str.strip().str.lower()
        if row_values.eq(block_label.lower()).any():
            match_row = idx
            break

    if match_row is None:
        return pd.DataFrame()

    header_row = match_row + 1
    if header_row >= raw_data.shape[0]:
        return pd.DataFrame()

    # Copy everything below the header
    df = raw_data.iloc[header_row:].copy()
    df.columns = raw_data.iloc[header_row].astype(str).str.strip()
    df = df.iloc[1:]  # skip header row itself

    # Standardize the date column name if needed
    if "Date Or Hour" in df.columns and "Date" not in df.columns:
        df.rename(columns={"Date Or Hour": "Date"}, inplace=True)

    if "Date" not in df.columns:
        return pd.DataFrame()

    df["Date_parsed"] = pd.to_datetime(df["Date"], errors="coerce")
    df = df[df["Date_parsed"].notna()].copy()

    return df


daily_dataframes = []
hourly_dataframes = []

for sheet_name in workbook.sheet_names:
    sheet_df = workbook.parse(sheet_name, header=None)

    daily_df = extract_block(sheet_df, "Daily Data")
    hourly_df = extract_block(sheet_df, "Hourly Data")

    for dframe in (daily_df, hourly_df):
        if dframe.empty:
            continue
        dframe.columns = [str(c).strip() for c in dframe.columns]
        dframe["app_id"] = sheet_name
        dframe["status"] = "Valid" if "Valid" in sheet_name else "Invalid"

    if not daily_df.empty:
        daily_dataframes.append(daily_df)
    if not hourly_df.empty:
        hourly_dataframes.append(hourly_df)

# Combine results
daily_master = pd.concat(daily_dataframes, ignore_index=True) if daily_dataframes else pd.DataFrame()
hourly_master = pd.concat(hourly_dataframes, ignore_index=True) if hourly_dataframes else pd.DataFrame()

# Clean up numeric columns
num_cols = [
    "unique_idfas", "unique_ips", "unique_uas", "total_requests",
    "requests_per_idfa", "impressions", "impressions_per_idfa",
    "idfa_ip_ratio", "idfa_ua_ratio", "IVT"
]

for col in num_cols:
    if col in daily_master.columns:
        daily_master[col] = pd.to_numeric(daily_master[col], errors="coerce")
    if col in hourly_master.columns:
        hourly_master[col] = pd.to_numeric(hourly_master[col], errors="coerce")

print(f"Daily rows: {len(daily_master)} | Hourly rows: {len(hourly_master)}")


Daily rows: 534 | Hourly rows: 504


Cleaning up and filtering to true Daily Rows

In [3]:
# Remove stray NaN or unnamed columns
for frame in (daily_master, hourly_master):
    drop_cols = [
        c for c in frame.columns 
        if str(c).strip().lower() in ('nan', 'unnamed: 0', 'unnamed:0', '')
    ]
    if drop_cols:
        frame.drop(columns=drop_cols, inplace=True, errors="ignore")

# Some daily sheets have mixed hourly data — filter for true daily (00:00)
midnight_time = pd.to_datetime("00:00:00").time()
daily_clean = daily_master[daily_master["Date_parsed"].dt.time == midnight_time].copy()
hourly_clean = hourly_master.copy()

# Save cleaned data
output_dir = Path("outputs")
daily_path = output_dir / "daily_master_clean.csv"
hourly_path = output_dir / "hourly_master_clean.csv"

daily_clean.to_csv(daily_path, index=False)
hourly_clean.to_csv(hourly_path, index=False)

print(f"Saved cleaned datasets: {len(daily_clean)} daily rows, {len(hourly_clean)} hourly rows.")
if "app_id" in daily_clean.columns:
    apps = daily_clean["app_id"].unique().tolist()
    print(f"Apps included: {apps}")


Saved cleaned datasets: 54 daily rows, 504 hourly rows.
Apps included: ['Valid 1', 'Valid 2', 'Valid 3', 'Invalid 1', 'Invalid 2', 'Invalid 3']


EDA : Summary Stats by App

In [4]:
# Generate summary statistics grouped by app and status
summary = (
    daily_clean.groupby(["app_id", "status"])[
        [
            "unique_idfas", "unique_ips", "unique_uas",
            "idfa_ua_ratio", "idfa_ip_ratio",
            "requests_per_idfa", "IVT"
        ]
    ]
    .agg(["median", "mean", "std"])
    .reset_index()
)

# Flatten multi-level column headers for easier CSV export
summary.columns = ["_".join(col).strip("_") for col in summary.columns.to_flat_index()]

print("\nSummary Stats by App")
print(summary.to_string(index=False))

# Save results
output_file = Path("outputs") / "summary_by_app.csv"
summary.to_csv(output_file, index=False)

print(f"\nSummary saved to: {output_file}")



Summary Stats by App
   app_id  status  unique_idfas_median  unique_idfas_mean  unique_idfas_std  unique_ips_median  unique_ips_mean  unique_ips_std  unique_uas_median  unique_uas_mean  unique_uas_std  idfa_ua_ratio_median  idfa_ua_ratio_mean  idfa_ua_ratio_std  idfa_ip_ratio_median  idfa_ip_ratio_mean  idfa_ip_ratio_std  requests_per_idfa_median  requests_per_idfa_mean  requests_per_idfa_std  IVT_median  IVT_mean  IVT_std
Invalid 1 Invalid              55073.0      120811.666667     128745.976541            55062.0    120747.333333   128653.852368              222.0       219.000000       42.311346            248.076577          482.229666         465.002180              1.000196            1.000275           0.000288                  1.093730                1.108404               0.082892    0.994737  0.751958 0.431354
Invalid 2 Invalid              28822.0       63570.222222      67748.358544            28819.0     63527.777778    67688.343826              834.0       864.111111   

Finding first IVT spike per app (hourly)

In [5]:
# Sort hourly data by app and timestamp
hourly_sorted = hourly_clean.sort_values(["app_id", "Date_parsed"])

# Identify first occurrence where IVT exceeds 0.5
first_ivt_high = (
    hourly_sorted[hourly_sorted["IVT"] > 0.5]
    .groupby("app_id", as_index=False)["Date_parsed"]
    .min()
    .rename(columns={"Date_parsed": "first_high_IVT"})
)

print("\nFirst High IVT (>0.5) Timestamp")
print(first_ivt_high.to_string(index=False))

# Save the results
ivt_output_path = Path("outputs") / "first_high_IVT_per_app.csv"
first_ivt_high.to_csv(ivt_output_path, index=False)
print(f"\nSaved: {ivt_output_path}")

# Check which apps never crossed the IVT threshold
all_apps = pd.DataFrame({"app_id": daily_clean["app_id"].unique()})
ivt_flags = all_apps.merge(first_ivt_high, on="app_id", how="left")

# Classify apps based on their first IVT breach timing
def classify_flag(ts):
    if pd.isna(ts):
        return "Never flagged"
    ts_date = pd.to_datetime(ts)
    cutoff = pd.to_datetime("2025-09-12")
    if ts_date <= cutoff:
        return "Early"
    return "Delayed"

ivt_flags["flag_type"] = ivt_flags["first_high_IVT"].apply(classify_flag)

print("\nFlag Classification")
print(ivt_flags[["app_id", "first_high_IVT", "flag_type"]].to_string(index=False))



First High IVT (>0.5) Timestamp
   app_id      first_high_IVT
Invalid 1 2025-09-12 05:00:00
Invalid 2 2025-09-11 21:00:00
Invalid 3 2025-09-13 05:00:00

Saved: outputs\first_high_IVT_per_app.csv

Flag Classification
   app_id      first_high_IVT     flag_type
  Valid 1                 NaT Never flagged
  Valid 2                 NaT Never flagged
  Valid 3                 NaT Never flagged
Invalid 1 2025-09-12 05:00:00       Delayed
Invalid 2 2025-09-11 21:00:00         Early
Invalid 3 2025-09-13 05:00:00       Delayed


Before and after comparison for Invalid apps

In [8]:
results = []

# Iterate through each app’s first high IVT record
for _, row in ivt_flags.iterrows():
    app_id = row["app_id"]
    spike_time = row["first_high_IVT"]

    if pd.isna(spike_time):  # skip apps that never crossed the threshold
        continue

    app_df = daily_clean[daily_clean["app_id"] == app_id].sort_values("Date_parsed")
    before_spike = app_df[app_df["Date_parsed"] < spike_time]
    after_spike = app_df[app_df["Date_parsed"] >= spike_time]

    def calc_stats(sub):
        return {
            "rows": len(sub),
            "mean_unique_idfas": sub["unique_idfas"].mean(),
            "mean_unique_uas": sub["unique_uas"].mean(),
            "mean_requests_per_idfa": sub["requests_per_idfa"].mean(),
            "mean_idfa_ua_ratio": sub["idfa_ua_ratio"].mean(),
            "mean_idfa_ip_ratio": sub["idfa_ip_ratio"].mean(),
            "mean_IVT": sub["IVT"].mean()
        }

    results.append({
        "app_id": app_id,
        "first_high_IVT": spike_time,
        "before": calc_stats(before_spike),
        "after": calc_stats(after_spike)
    })

print("\nBefore / After IVT Spike Comparison")
for r in results:
    print(f"\nApp: {r['app_id']}  |  Spike at: {r['first_high_IVT']}")
    print(f"  Days before: {r['before']['rows']}  |  After: {r['after']['rows']}")
    print(f"  idfa_ua_ratio: {r['before']['mean_idfa_ua_ratio']:.2f} → {r['after']['mean_idfa_ua_ratio']:.2f} "
          f"({r['after']['mean_idfa_ua_ratio']/r['before']['mean_idfa_ua_ratio']:.2f}×)")
    print(f"  requests_per_idfa: {r['before']['mean_requests_per_idfa']:.2f} → {r['after']['mean_requests_per_idfa']:.2f} "
          f"({r['after']['mean_requests_per_idfa']/r['before']['mean_requests_per_idfa']:.2f}×)")
    print(f"  IVT: {r['before']['mean_IVT']:.3f} → {r['after']['mean_IVT']:.3f}")



Before / After IVT Spike Comparison

App: Invalid 1  |  Spike at: 2025-09-12 05:00:00
  Days before: 3  |  After: 6
  idfa_ua_ratio: 622.60 → 412.05 (0.66×)
  requests_per_idfa: 1.12 → 1.10 (0.99×)
  IVT: 0.261 → 0.998

App: Invalid 2  |  Spike at: 2025-09-11 21:00:00
  Days before: 1  |  After: 8
  idfa_ua_ratio: 33.66 → 62.44 (1.86×)
  requests_per_idfa: 1.04 → 1.09 (1.05×)
  IVT: 0.274 → 0.997

App: Invalid 3  |  Spike at: 2025-09-13 05:00:00
  Days before: 5  |  After: 4
  idfa_ua_ratio: 485.11 → 275.50 (0.57×)
  requests_per_idfa: 1.21 → 1.12 (0.93×)
  IVT: 0.162 → 0.995


Visualizing IVT over time per app

In [11]:
fig_dir = Path('figures')

for app in hourly_clean['app_id'].unique():
    df_app = hourly_clean[hourly_clean['app_id'] == app].sort_values('Date_parsed')
    
    plt.figure(figsize=(12, 3))
    plt.plot(df_app['Date_parsed'], df_app['IVT'], marker='o', markersize=3, linestyle='-', linewidth=1)
    plt.axhline(0.5, color='red', linestyle='--', linewidth=1, label='IVT threshold (0.5)')
    plt.title(f'IVT over Time — {app}', fontsize=12, fontweight='bold')
    plt.xlabel('Date/Hour')
    plt.ylabel('IVT')
    plt.ylim(-0.05, 1.05)
    plt.legend()
    plt.tight_layout()
    plt.savefig(fig_dir / f'IVT_timeline_{app.replace(" ", "_")}.png', dpi=150)
    plt.close()

print(f"Saved IVT timeline plots to figures/")


Saved IVT timeline plots to figures/


Comparing Valid vs Invalid - key distribution ratios

In [12]:
import pandas as pd

dm = pd.read_csv('outputs/daily_master_clean.csv', parse_dates=['Date_parsed'])
fh = pd.read_csv('outputs/first_high_IVT_per_app.csv', parse_dates=['first_high_IVT'])
sb = pd.read_csv('outputs/summary_by_app.csv')

metrics = ['IVT','idfa_ua_ratio','idfa_ip_ratio','requests_per_idfa','unique_idfas','unique_uas']
rows = []
for _, r in fh.iterrows():
    app, t = r['app_id'], r['first_high_IVT']
    dfa = dm[dm['app_id']==app].sort_values('Date_parsed')
    before, after = dfa[dfa['Date_parsed']<t], dfa[dfa['Date_parsed']>=t]
    if before.empty or after.empty: 
        continue
    for m in metrics:
        b, a = before[m].mean(), after[m].mean()
        rows.append({'app_id': app, 'first_high_IVT': t, 'metric': m,
                     'before_mean': b, 'after_mean': a,
                     'delta_abs': a-b, 'delta_ratio': (a/(b if b!=0 else 1))})

deltas = pd.DataFrame(rows)
deltas.to_csv('outputs/before_after_deltas.csv', index=False)

apps = sb[['app_id','status']].drop_duplicates()
flags = apps.merge(fh, on='app_id', how='left')
flags['flag_type'] = flags['first_high_IVT'].apply(
    lambda x: 'Never flagged' if pd.isna(x) else 'Early' if pd.to_datetime(x) <= pd.to_datetime('2025-09-12') else 'Delayed'
)
flags.to_csv('outputs/app_flag_summary.csv', index=False)
print('Saved before_after_deltas.csv and app_flag_summary.csv')


Saved before_after_deltas.csv and app_flag_summary.csv


Export Deltas + Summary Tables

In [13]:
import pandas as pd

dm = pd.read_csv('outputs/daily_master_clean.csv', parse_dates=['Date_parsed'])
fh = pd.read_csv('outputs/first_high_IVT_per_app.csv', parse_dates=['first_high_IVT'])
sb = pd.read_csv('outputs/summary_by_app.csv')

metrics = ['IVT','idfa_ua_ratio','idfa_ip_ratio','requests_per_idfa','unique_idfas','unique_uas']
rows = []
for _, r in fh.iterrows():
    app, t = r['app_id'], r['first_high_IVT']
    dfa = dm[dm['app_id']==app].sort_values('Date_parsed')
    before, after = dfa[dfa['Date_parsed']<t], dfa[dfa['Date_parsed']>=t]
    if before.empty or after.empty: 
        continue
    for m in metrics:
        b, a = before[m].mean(), after[m].mean()
        rows.append({'app_id': app, 'first_high_IVT': t, 'metric': m,
                     'before_mean': b, 'after_mean': a,
                     'delta_abs': a-b, 'delta_ratio': (a/(b if b!=0 else 1))})

deltas = pd.DataFrame(rows)
deltas.to_csv('outputs/before_after_deltas.csv', index=False)

apps = sb[['app_id','status']].drop_duplicates()
flags = apps.merge(fh, on='app_id', how='left')
flags['flag_type'] = flags['first_high_IVT'].apply(
    lambda x: 'Never flagged' if pd.isna(x) else 'Early' if pd.to_datetime(x) <= pd.to_datetime('2025-09-12') else 'Delayed'
)
flags.to_csv('outputs/app_flag_summary.csv', index=False)
print('Saved before_after_deltas.csv and app_flag_summary.csv')


Saved before_after_deltas.csv and app_flag_summary.csv
